In [2]:
import os
import re
import math
import cv2
import pandas as pd
import numpy as np

from tqdm import tqdm
from tools.settings import *
from keras.models import Sequential
from keras.preprocessing import image
from keras.layers import Dense, Dropout
from keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import to_categorical
from tools.train_val_test_spliter import split

In [112]:
# To split dataset if already splits folder already exits no need to run it
split()

Splitting the ginen dataset into Train Test=0.4 Validation=0.1
Done


In [10]:
train = pd. read_csv(os.path.join(dataset_path, "train.csv"))
test = pd. read_csv(os.path.join(dataset_path, "test.csv"))
val = pd. read_csv(os.path.join(dataset_path, "val.csv"))

In [3]:
train.head()

,Video_url,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch


In [4]:
test.head()

,Video_url,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap


In [11]:
val.head()

,Video_url,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap


In [12]:
print(train.shape)
print(test.shape)
print(val.shape)

(143, 2)
(111, 2)
(27, 2)


In [13]:
def convert_to_frame(data, folder_name):
    '''
    Generated filenames format dataset_path/folder_name/video_name_frame{number}_action.jpg
    '''
    directory = os.path.join(dataset_path, folder_name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    for i in tqdm(range(data.shape[0])):
        video_file = data['Video_url'][i]
        action = data['action'][i]
        video_name_list = video_file.split('/')[-1].split('.')
        video_name_list = video_name_list[:-1]
        video_name = ""
        for n in video_name_list:
            video_name += n
        # capturing the video from the given path
        capture = cv2.VideoCapture(video_file) 
        #frame rate
        frame_rate = capture.get(5)
        count = 0
        while(capture.isOpened()):
            #current frame number
            frame_id = capture.get(1) 
            read_correctly, frame = capture.read()
            if not read_correctly:
                break
            if (frame_id % math.floor(frame_rate) == 0):
                # storing the frames in a new folder named train_1
                filename = directory + "/" + video_name + "_frame{}_".format(count) + action +".jpg"
                count += 1
                cv2.imwrite(filename, frame)
        capture.release()
    print("Successfully Converted")

In [89]:
convert_to_frame(train, train_frames_path_name)

100%|██████████| 143/143 [00:50<00:00,  2.84it/s]

Successfully Converted


In [15]:
convert_to_frame(val, val_frames_path_name)

100%|██████████| 27/27 [00:09<00:00,  2.89it/s]

Successfully Converted


In [17]:
def create_paths_csv(directory, file_name):
    images = os.listdir(directory)
    images_path_list = []
    images_action_list = [] 
    for image in images:
        images_path_list.append(directory + image)
        images_action_list.append(image.split('.')[0].split('_')[-1])
    df = pd.DataFrame()
    df['image'] = images_path_list
    df['action'] = images_action_list
    print(os.path.join(dataset_path, file_name+'.csv'))
    df.to_csv(os.path.join(dataset_path, file_name+'.csv'), index=False)

In [121]:
create_paths_csv(train_frames_path, train_frames_path_name)

/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-time-action-recognition-from-video-footage/dataset/train_frames.csv


In [18]:
create_paths_csv(val_frames_path, val_frames_path_name)

/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-time-action-recognition-from-video-footage/dataset/val_frames.csv


In [3]:
train_image = pd.read_csv(os.path.join(dataset_path, 'train_frames.csv'))
train_image.head()

,image,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap


In [4]:
print(train_image.shape)

(620, 2)


In [5]:
val_image = pd.read_csv(os.path.join(dataset_path, 'val_frames.csv'))
val_image.head()

,image,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap


In [6]:
print(val_image.shape)

(115, 2)


In [11]:
action_nums = {"action": {"slap": 0, "kick": 1, "punch": 3}}

In [12]:
train_image = train_image.replace(action_nums)
train_image.head()

,image,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,3
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0


In [13]:
val_image = val_image.replace(action_nums)
val_image.head()

,image,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,1
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,0


In [14]:
def convert_to_array_and_split(image_data):
    image_value = []
    for i in tqdm(range(image_data.shape[0])):
        img = image.load_img(image_data['image'][i], target_size=(224,224,3))
        img = image.img_to_array(img)
        # normalizing the pixel value
        img = img / 255
        image_value.append(img)

    X = np.array(image_value)
    y = image_data['action']
    return X, y

In [15]:
X_train, y_train = convert_to_array_and_split(train_image)
print(X_train.shape)

100%|██████████| 620/620 [00:07<00:00, 85.03it/s]


(620, 224, 224, 3)


In [16]:
X_val, y_val = convert_to_array_and_split(val_image)
print(X_val.shape)

100%|██████████| 115/115 [00:01<00:00, 90.46it/s]


(115, 224, 224, 3)


In [17]:
'''This model was trained on a dataset that has 1,000 classes. 
include_top = False will remove the last layer of this model so that we can tune it as per our need.
'''
base_model = VGG16(weights='imagenet', include_top=False)

In [18]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(620, 7, 7, 512)

In [19]:
X_val = base_model.predict(X_val)
X_val.shape

(115, 7, 7, 512)

In [20]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(620, 7*7*512)
X_val = X_val.reshape(115, 7*7*512)

In [21]:
# normalizing the pixel values
max_pixel = X_train.max()
X_train = X_train / max_pixel
X_val = X_val / max_pixel
print(X_train.shape)
print(X_val.shape)

(620, 25088)
(115, 25088)


In [22]:
# y_train.value_counts()
y_train = to_categorical(y_val, 3)
y_val = to_categorical(y_val, 3)
print(y_train)

IndexError: index 3 is out of bounds for axis 1 with size 3

In [23]:
# The input shape will be 25,088
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               3

In [24]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_weight = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [25]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[mcp_weight], batch_size=128)

Epoch 1/50
5/5 [==============================] - 2s 302ms/step - loss: 0.0000e+00 - accuracy: 0.2102 - val_loss: 0.0000e+00 - val_accuracy: 0.2174
Epoch 2/50
5/5 [==============================] - 1s 219ms/step - loss: 0.0000e+00 - accuracy: 0.2299 - val_loss: 0.0000e+00 - val_accuracy: 0.2174
Epoch 3/50
5/5 [==============================] - 1s 218ms/step - loss: 0.0000e+00 - accuracy: 0.2337 - val_loss: 0.0000e+00 - val_accuracy: 0.2174
Epoch 4/50
5/5 [==============================] - 1s 245ms/step - loss: 0.0000e+00 - accuracy: 0.2316 - val_loss: 0.0000e+00 - val_accuracy: 0.2174
Epoch 5/50
5/5 [==============================] - 1s 219ms/step - loss: 0.0000e+00 - accuracy: 0.2321 - val_loss: 0.0000e+00 - val_accuracy: 0.2174
Epoch 6/50
5/5 [==============================] - 1s 237ms/step - loss: 0.0000e+00 - accuracy: 0.2221 - val_loss: 0.0000e+00 - val_accuracy: 0.2174
Epoch 7/50
5/5 [==============================] - 1s 239ms/step - loss: 0.0000e+00 - accuracy: 0.2467 - val_loss

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               3